In [1]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

import os
from IPython.display import clear_output

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = []


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
setup_env()
clear_output()
print("🚀 Setup complete. Continue to the next cell.")

🚀 Setup complete. Continue to the next cell.


In [6]:
# Automated Ollama installer script
# This script installs Ollama on Linux and configures it as a startup service.

import platform
import subprocess
import traceback


def run_shell_command(command: str) -> None:
    """Runs a shell command."""
    try:
        output = subprocess.run(command, shell=True, check=True)
        if output.returncode == 0:
            print(f"Command '{command}' executed successfully.")
        else:
            print(f"Command '{command}' failed to execute.")
    except Exception as e:
        print(f"Command '{command}' failed to execute due to unexpected error.")
        traceback.print_exc()


def check_ollama():
    """Checks if Ollama is installed. If not, installs it."""
    try:
        run_shell_command("ollama --version")
        print("Ollama is already installed.")
    except:
        print("Ollama is not installed. Installing Ollama!")
        install_ollama()


def install_ollama():
    """Installs Ollama based on the platform."""
    system = platform.system().lower()
    if system == "linux":
        setup_ollama_linux()
    elif system == "darwin":
        print("MacOS installation is not implemented yet.")
    elif system == "windows":
        print("Windows installation is not implemented yet.")
    else:
        print(f"Unsupported platform: {system}")


def setup_ollama_linux():
    """Installs and configures Ollama on Linux."""
    print("Starting Linux installation.")
    # Remove old libraries if upgrading
    run_shell_command("sudo rm -rf /usr/lib/ollama")

    # Download and extract the package
    run_shell_command(
        "curl -L https://ollama.com/download/ollama-linux-amd64.tgz -o ollama-linux-amd64.tgz"
    )
    run_shell_command("sudo tar -C /usr -xzf ollama-linux-amd64.tgz")

    # Optional: Install AMD GPU or ARM64 package based on user choice
    choice = (
        input("Do you have an AMD GPU or need ARM64 support? (amd/arm64/none): ")
        .strip()
        .lower()
    )
    if choice == "amd":
        run_shell_command(
            "curl -L https://ollama.com/download/ollama-linux-amd64-rocm.tgz -o ollama-linux-amd64-rocm.tgz"
        )
        run_shell_command("sudo tar -C /usr -xzf ollama-linux-amd64-rocm.tgz")
    elif choice == "arm64":
        run_shell_command(
            "curl -L https://ollama.com/download/ollama-linux-arm64.tgz -o ollama-linux-arm64.tgz"
        )
        run_shell_command("sudo tar -C /usr -xzf ollama-linux-arm64.tgz")

    # Add Ollama as a startup service
    configure_startup_service()

    # Start Ollama service
    run_shell_command("sudo systemctl start ollama")
    run_shell_command("sudo systemctl status ollama")

    print("Ollama installation and configuration completed.")


def configure_startup_service():
    """Configures Ollama as a startup service."""
    print("Configuring Ollama as a startup service.")

    # Create user and group for Ollama
    run_shell_command("sudo useradd -r -s /bin/false -U -m -d /usr/share/ollama ollama")
    run_shell_command("sudo usermod -a -G ollama $(whoami)")

    # Create systemd service file
    service_file_content = """[Unit]
Description=Ollama Service
After=network-online.target

[Service]
ExecStart=/usr/bin/ollama serve
User=ollama
Group=ollama
Restart=always
RestartSec=3
Environment=\"PATH=$PATH\"

[Install]
WantedBy=default.target
"""

    with open("/etc/systemd/system/ollama.service", "w") as f:
        f.write(service_file_content)

    run_shell_command("sudo systemctl daemon-reload")
    run_shell_command("sudo systemctl enable ollama")


if __name__ == "__main__":
    check_ollama()

In [4]:
check_ollama()

ollama version is 0.3.11
Command 'ollama --version' executed successfully.
